In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

from os.path import isfile, exists
from os import makedirs, open, O_WRONLY, dup2

INIT_CAP = 1000

In [12]:
def fetch_data(symbols, start_date, end_date) -> pd.DataFrame:
    file_name = "_".join(symbols + [start_date, end_date])
    
    data = yf.download(symbols, start=start_date, end=end_date)["Adj Close"]
    
    if not exists("./data"):
        makedirs("./data")

    if not data.empty:
        data.to_pickle(f"./data/{file_name}.pkl")

    return data

In [13]:
def get_data(symbols, start, end) -> pd.DataFrame:
    symbols.sort()
    file_name = "_".join(symbols + [start, end])

    if not isfile(f"./data/{file_name}.pkl"):
        fetch_data(symbols, start, end)
        print()

    data = pd.read_pickle(f"./data/{file_name}.pkl")

    return data

In [14]:
def make_combo(capital: int, stock_data: pd.DataFrame, weights) -> pd.Series:
    initial_prices = stock_data.iloc[0]
    units = (capital * weights) / initial_prices
    values = stock_data.dot(units)

    return values

In [15]:
def net_profit(values: pd.Series, verbose=False):
    profit = values[-1] - values[0]
    pct = 100 * (profit / values[0])

    if verbose:
        print(f"Net Profit:\t{profit:.2f}")
        print(f"           \t{pct:.2f} %")
        print("-" * 80)

    return profit


def sharpe_ratio(values: pd.Series, verbose=False):
    returns = values.pct_change().dropna() * 100

    # NASDAQ risk-free rate
    rf_annual = 3.9

    rp = np.mean(returns)
    returns_std = np.std(returns)

    sharpe = (rp - (rf_annual / 252)) / returns_std

    annual_sharpe = sharpe * np.sqrt(252)

    if verbose:
        print(f"Sharpe Ratio:\t{annual_sharpe:.4f}")
        print("-" * 80)

    return annual_sharpe


def sortino_ratio(values: pd.Series, verbose=False):
    returns = values.pct_change().dropna() * 100

    # NASDAQ risk-free rate
    rf_annual = 3.9

    rp = np.mean(returns)
    loss_returns_std = np.std(returns[returns < 0])

    sortino = (rp - (rf_annual / 252)) / loss_returns_std

    annual_sortino = sortino * np.sqrt(252)

    if verbose:
        print(f"Sortino Ratio:\t{annual_sortino:.4f}")
        print("-" * 80)

    return annual_sortino

In [16]:
def test_weights_on_interval(data, weights, start, end, title=""):
    portfolio_value = make_combo(INIT_CAP, data, weights)

    print()
    print(f"{title} Buy & Hold status over {start} to {end}:")
    print("-" * 80)
    print("Weights:", *[f"{w:.2f}" for w in weights], sep="\t")
    print("-" * 80)

    net_profit(portfolio_value, verbose=True)
    sharpe_ratio(portfolio_value, verbose=True)
    sortino_ratio(portfolio_value, verbose=True)

In [17]:
def optimize_weights(data, verbose=False):
    def optimize_target(obj_func, title, verbose=False):
        initial_weights = np.ones(4) / 4

        constraints = (
            {"type": "eq", "fun": lambda w: np.sum(w) - 1},
            {"type": "ineq", "fun": lambda w: w[0]},
            {"type": "ineq", "fun": lambda w: w[1]},
            {"type": "ineq", "fun": lambda w: w[2]},
            {"type": "ineq", "fun": lambda w: w[3]},
        )

        result_obj = minimize(
            obj_func, initial_weights, method="SLSQP", constraints=constraints
        )

        opt_weights = result_obj.x
        if verbose:
            print(f"Optimized Weights for {title}:", opt_weights)

        return opt_weights

    def objective_net_profit(weights):
        portfolio_value = make_combo(INIT_CAP, data, weights)

        # Negative because we want to maximize
        return -net_profit(portfolio_value)

    def objective_sharpe_ratio(weights):
        portfolio_value = make_combo(INIT_CAP, data, weights)

        # Negative because we want to maximize
        return -sharpe_ratio(portfolio_value)

    def objective_sortino_ratio(weights):
        portfolio_value = make_combo(INIT_CAP, data, weights)

        # Negative because we want to maximize
        return -sortino_ratio(portfolio_value)

    if verbose: print()
    np_weights = optimize_target(objective_net_profit, "Net Profit", verbose)
    sh_weights = optimize_target(objective_sharpe_ratio, "Sharpe Ratio", verbose)
    so_weights = optimize_target(objective_sortino_ratio, "Sortino Ratio", verbose)

    return np_weights, sh_weights, so_weights

In [18]:
def do_all_for(symbols, init_coeffs=None):
    start = "2022-11-01"
    end = "2023-11-01"

    data = get_data(symbols, start, end)

    print("Symbols:    ", *symbols, sep="    ")

    if init_coeffs is None:
        init_coeffs = np.random.rand(4)
        init_coeffs /= init_coeffs.sum()

    test_weights_on_interval(data, init_coeffs, start, end, "Initial")

    np_opt1, sh_opt1, so_opt1 = optimize_weights(data, verbose=True)

    test_weights_on_interval(data, np_opt1, start, end, "Net_Profit-optimized")
    test_weights_on_interval(data, sh_opt1, start, end, "Sharpe_Ratio-optimized")
    test_weights_on_interval(data, so_opt1, start, end, "Sortino_Ratio-optimized")

    new_start = "2023-11-02"
    new_end = "2023-12-02"
    new_data = get_data(symbols, new_start, new_end)

    test_weights_on_interval(new_data, init_coeffs, new_start, new_end, "Initial")

    test_weights_on_interval(
        new_data, np_opt1, new_start, new_end, "Previously Net_Profit-optimized"
    )
    test_weights_on_interval(
        new_data, sh_opt1, new_start, new_end, "Previously Sharpe_Ratio-optimized"
    )
    test_weights_on_interval(
        new_data, so_opt1, new_start, new_end, "Previously Sortino_Ratio-optimized"
    )

    print()
    np_opt2, sh_opt2, so_opt2 = optimize_weights(new_data, verbose=True)

    test_weights_on_interval(
        new_data, np_opt2, new_start, new_end, "New Net_Profit-optimized"
    )
    test_weights_on_interval(
        new_data, sh_opt2, new_start, new_end, "New Sharpe_Ratio-optimized"
    )
    test_weights_on_interval(
        new_data, so_opt2, new_start, new_end, "New Sortino_Ratio-optimized"
    )


In [19]:
symbols = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'ADA-USD']

# init_coeffs = np.array([0.7, 0.15, 0.1, 0.05])
init_coeffs = np.random.rand(4)
init_coeffs /= init_coeffs.sum()

In [20]:
do_all_for(symbols, init_coeffs)

[*********************100%%**********************]  4 of 4 completed

Symbols:        ADA-USD    BTC-USD    ETH-USD    XRP-USD

Initial Buy & Hold status over 2022-11-01 to 2023-11-01:
--------------------------------------------------------------------------------
Weights:	0.34	0.37	0.24	0.06
--------------------------------------------------------------------------------
Net Profit:	217.23
           	21.72 %
--------------------------------------------------------------------------------
Sharpe Ratio:	0.4410
--------------------------------------------------------------------------------
Sortino Ratio:	0.6167
--------------------------------------------------------------------------------

Optimized Weights for Net Profit: [-1.85286524e-08  1.00000002e+00 -2.53137955e-09  2.88778779e-09]
Optimized Weights for Sharpe Ratio: [-9.71445158e-17  1.00000000e+00 -6.10622664e-16 -3.33066907e-16]
Optimized Weights for Sortino Ratio: [-6.93889390e-16  1.00000000e+00 -2.03135278e-15  5.5511151